# ANN- Univariate Time Series Solar Radiation(GHI) Forecasting with MLP, CNN, and LSTM Networks

# MLP

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv

from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load and preprocess the dataset
#data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')
data = read_csv('Palestine-PV.csv', parse_dates=['date'],  index_col='date')

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Sort the DataFrame by the index (date) in ascending order to ensure oldest to newest
data = data.sort_index()

# Manually specify column names
# Now select the first column after the index
data = data.iloc[:, 5] 

data = data[~np.isnan(data)]
data = data.dropna()

data = data.astype('float32')

# Convert the Series to a numpy array and reshape
data = np.array(data).reshape(-1, 1)

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split into train and test sets
train_size = int(len(scaled_data) * 0.80)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]

def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])
        
    return np.array(x), np.array(y)

seq_size = 3  # Number of time steps to look back 

train_X, train_y = to_sequences(train, seq_size)
test_X, test_y = to_sequences(test, seq_size)
train_X, train_y = torch.tensor(train_X).float(), torch.tensor(train_y).view(-1, 1).float()
test_X, test_y = torch.tensor(test_X).float(), torch.tensor(test_y).view(-1, 1).float()
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)  # Corrected to print test_y.shape

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))
        self.layers.append(nn.Linear(neurons, 1))
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Function to count the trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Parameters
activation = 'ReLU'
batch_size = 64
dropout_rate = 0.05
epochs = 60
layers1 = 2
learning_rate = 0.001
neurons = 32
input_dim = train_X.shape[1]

# Prepare the data
train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # Changed shuffle to False for validation

# Initialize the model, loss function, and optimizer
model = NeuralNetwork(input_dim, neurons, layers1, activation, dropout_rate)
model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Print trainable parameters
print(f"Trainable parameters: {count_parameters(model)}")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.numel()} parameters")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
train_losses = []
val_losses = []
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(model.device if hasattr(model, 'device') else torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        batch_y = batch_y.to(model.device if hasattr(model, 'device') else torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for val_X, val_y in test_loader:
            val_X = val_X.to(model.device if hasattr(model, 'device') else torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            val_y = val_y.to(model.device if hasattr(model, 'device') else torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            val_outputs = model(val_X)
            val_loss += criterion(val_outputs, val_y).item()
    val_loss /= len(test_loader)
    train_losses.append(loss.item())
    val_losses.append(val_loss)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}, Val Loss: {val_loss:.6f}')
    if val_loss < min_val_loss:
        min_val_loss = val_loss

# Print the minimum validation loss
print(f"Minimum Validation Loss (MSE): {min_val_loss:.6f}")

# Compute the square of the minimum validation loss
squared_min_val_loss = np.sqrt(min_val_loss )
print(f"Square of Minimum Validation Loss: {squared_min_val_loss:.6f}")

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(test_X.to(model.device if hasattr(model, 'device') else torch.device('cuda' if torch.cuda.is_available() else 'cpu'))).cpu().numpy()

# Inverse transform the scaled predictions and true values to original scale
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

# Evaluate the model
mse = criterion(torch.tensor(predictions), test_y).item()
print("Mean Squared Error (MSE): {:.6f}".format(mse))

# Calculate RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE): {:.6f}".format(rmse))

# Calculate Unnormalized MAE
mae = mean_absolute_error(test_y_unnorm, predictions_unnorm)
print("Unnormalized Mean Absolute Error (MAE): {:.6f}".format(mae))

# Calculate Unnormalized MSE
mse = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse=np.sqrt(mse)
print("Unnormalized Mean Sequare Error (MSE): {:.6f}".format(mse))
print("Unnormalized Root Mean Sequare Error (RMSE): {:.6f}".format(rmse))


torch.Size([21020, 3]) torch.Size([21020, 1]) torch.Size([5252, 3]) torch.Size([5252, 1])
Trainable parameters: 2273
layers.0.weight: 96 parameters
layers.0.bias: 32 parameters
layers.2.weight: 1024 parameters
layers.2.bias: 32 parameters
layers.5.weight: 1024 parameters
layers.5.bias: 32 parameters
layers.8.weight: 32 parameters
layers.8.bias: 1 parameters
Epoch 1/60, Loss: 0.007131, Val Loss: 0.004748
Epoch 2/60, Loss: 0.006280, Val Loss: 0.004035
Epoch 3/60, Loss: 0.008248, Val Loss: 0.003617
Epoch 4/60, Loss: 0.007108, Val Loss: 0.003482
Epoch 5/60, Loss: 0.003914, Val Loss: 0.003495
Epoch 6/60, Loss: 0.000936, Val Loss: 0.003557
Epoch 7/60, Loss: 0.021942, Val Loss: 0.003463
Epoch 8/60, Loss: 0.002667, Val Loss: 0.003475
Epoch 9/60, Loss: 0.005949, Val Loss: 0.003257
Epoch 10/60, Loss: 0.014127, Val Loss: 0.003131
Epoch 11/60, Loss: 0.007826, Val Loss: 0.003143
Epoch 12/60, Loss: 0.002152, Val Loss: 0.003623
Epoch 13/60, Loss: 0.002792, Val Loss: 0.003076
Epoch 14/60, Loss: 0.0011

In [ ]:
print(model)

# CNN

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv
from sklearn.metrics import mean_absolute_error, mean_squared_error  # Added for MAE and MSE

# Load and preprocess the dataset
#data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')
data = read_csv('Palestine-PV.csv', parse_dates=['date'],  index_col='date')

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Sort the DataFrame by the index (date) in ascending order to ensure oldest to newest
data = data.sort_index()

# Sort the DataFrame by the index (date) in ascending order to ensure oldest to newest
data = data.sort_index()

# Manually specify column names
# Now select the sixth column (index 5) after the index
data = data.iloc[:, 5] 

# Remove NaN values
data = data[~np.isnan(data)]
data = data.dropna()

data = data.astype('float32')

# Convert the Series to a numpy array and reshape
data = np.array(data).reshape(-1, 1)

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split into train and test sets
train_size = int(len(scaled_data) * 0.80)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset) - seq_size - 1):
        window = dataset[i:(i + seq_size), 0]
        x.append(window)
        y.append(dataset[i + seq_size, 0])
        
    return np.array(x), np.array(y)

seq_size = 3  # Number of time steps to look back 

train_X, train_y = to_sequences(train, seq_size)
test_X, test_y = to_sequences(test, seq_size)

train_X, train_y = torch.tensor(train_X).float(), torch.tensor(train_y).view(-1, 1).float()
test_X, test_y = torch.tensor(test_X).float(), torch.tensor(test_y).view(-1, 1).float()

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)  # Corrected to print test_y.shape

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=1)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=1)
        self.flatten = nn.Flatten()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim * 32, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))
        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

# Function to count the trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Parameters
activation = 'ReLU'
batch_size = 64
dropout_rate = 0.05
epochs = 60
layers1 = 2
learning_rate = 0.001
neurons = 32
input_dim = train_X.shape[1]

# Prepare the data
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))  # (batch_size, channels, seq_length)
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # Changed shuffle to False for validation

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, loss function, and optimizer
model = NeuralNetwork(input_dim, neurons, layers1, activation, dropout_rate)
model = model.to(device)

# Print trainable parameters
print(f"Trainable parameters: {count_parameters(model)}")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.numel()} parameters")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
train_losses = []
val_losses = []
min_val_loss = float('inf')
start_time = time.time()

for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)
            val_outputs = model(val_X)
            val_loss += criterion(val_outputs, val_y).item()
    val_loss /= len(test_loader)
    train_losses.append(loss.item())
    val_losses.append(val_loss)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}, Val Loss: {val_loss:.6f}')
    if val_loss < min_val_loss:
        min_val_loss = val_loss

end_time = time.time()
elapsed_time = end_time - start_time
print(f"The training process took {elapsed_time:.2f} seconds.")

# Print the minimum validation loss
print(f"Minimum Validation Loss (MSE): {min_val_loss:.6f}")

# Compute the RMSE of the minimum validation loss
rmse_min_val_loss = np.sqrt(min_val_loss)
print(f"Root Mean Squared Error of Minimum Validation Loss: {rmse_min_val_loss:.6f}")

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(test_X.to(device)).cpu().numpy()

# Inverse transform the scaled predictions and true values to original scale
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

# Evaluate the model on unnormalized data
mse_unnorm = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse_unnorm = np.sqrt(mse_unnorm)
mae_unnorm = mean_absolute_error(test_y_unnorm, predictions_unnorm)

print("Mean Squared Error (MSE) on Test Set (Normalized): {:.6f}".format(mean_squared_error(test_y.numpy(), predictions)))
print("Root Mean Squared Error (RMSE) on Test Set (Normalized): {:.6f}".format(np.sqrt(mean_squared_error(test_y.numpy(), predictions))))
print("Mean Squared Error (MSE) on Test Set (Unnormalized): {:.6f}".format(mse_unnorm))
print("Root Mean Squared Error (RMSE) on Test Set (Unnormalized): {:.6f}".format(rmse_unnorm))
print("Mean Absolute Error (MAE) on Test Set (Unnormalized): {:.6f}".format(mae_unnorm))



torch.Size([21020, 3]) torch.Size([21020, 1]) torch.Size([5252, 3]) torch.Size([5252, 1])
Trainable parameters: 6369
conv1.weight: 32 parameters
conv1.bias: 32 parameters
conv2.weight: 1024 parameters
conv2.bias: 32 parameters
layers.0.weight: 3072 parameters
layers.0.bias: 32 parameters
layers.2.weight: 1024 parameters
layers.2.bias: 32 parameters
layers.5.weight: 1024 parameters
layers.5.bias: 32 parameters
output_layer.weight: 32 parameters
output_layer.bias: 1 parameters
Epoch 1/60, Loss: 0.009286, Val Loss: 0.004284
Epoch 2/60, Loss: 0.004711, Val Loss: 0.004110
Epoch 3/60, Loss: 0.004137, Val Loss: 0.003797
Epoch 4/60, Loss: 0.005937, Val Loss: 0.003277
Epoch 5/60, Loss: 0.008613, Val Loss: 0.004457
Epoch 6/60, Loss: 0.004802, Val Loss: 0.003541
Epoch 7/60, Loss: 0.007850, Val Loss: 0.003375
Epoch 8/60, Loss: 0.008295, Val Loss: 0.003197
Epoch 9/60, Loss: 0.003011, Val Loss: 0.003581
Epoch 10/60, Loss: 0.011912, Val Loss: 0.004669
Epoch 11/60, Loss: 0.008284, Val Loss: 0.003427
E

In [14]:
print(model)

NeuralNetwork(
  (conv1): Conv1d(1, 32, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): ModuleList(
    (0): Linear(in_features=576, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
  )
  (output_layer): Linear(in_features=100, out_features=1, bias=True)
)


# LSTM

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pandas import read_csv
from sklearn.metrics import mean_absolute_error, mean_squared_error  # Added for MAE and MSE

# Load and preprocess the dataset
#data = read_csv('jordan_pv.csv', parse_dates=['date'], index_col='date')
data = read_csv('Palestine-PV.csv', parse_dates=['date'],  index_col='date')

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Sort the DataFrame by the index (date) in ascending order to ensure oldest to newest
data = data.sort_index()

# Sort the DataFrame by the index (date) in ascending order to ensure oldest to newest
data = data.sort_index()
# Manually specify column names
# Now select the sixth column (index 5) after the index
data = data.iloc[:, 5]

# Remove NaN values
data = data[~np.isnan(data)]
data = data.dropna()

data = data.astype('float32')

# Convert the Series to a numpy array and reshape
data = np.array(data).reshape(-1, 1)

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Split into train and test sets
train_size = int(len(scaled_data) * 0.80)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset) - seq_size - 1):
        window = dataset[i:(i + seq_size), 0]
        x.append(window)
        y.append(dataset[i + seq_size, 0])

    return np.array(x), np.array(y)

seq_size = 3  # Number of time steps to look back 

train_X, train_y = to_sequences(train, seq_size)
test_X, test_y = to_sequences(test, seq_size)

train_X, train_y = torch.tensor(train_X).float(), torch.tensor(train_y).view(-1, 1).float()
test_X, test_y = torch.tensor(test_X).float(), torch.tensor(test_y).view(-1, 1).float()

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)  # Corrected to print test_y.shape

# Define the LSTM model
class LSTMNetwork(nn.Module):
    def __init__(self, input_dim, neurons, layers1, activation, dropout_rate):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_dim, neurons, batch_first=True)
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(neurons, neurons))
        self.layers.append(getattr(nn, activation)())
        for _ in range(layers1 - 1):
            self.layers.append(nn.Linear(neurons, neurons))
            self.layers.append(getattr(nn, activation)())
            if dropout_rate > 0:
                self.layers.append(nn.Dropout(p=dropout_rate))
        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # get the last output of the sequence
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

# Function to count the trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Parameters
activation = 'ReLU'
batch_size = 64
dropout_rate = 0.05
epochs = 60
layers1 = 2
learning_rate = 0.001
neurons = 32
input_dim = train_X.shape[1]

# Prepare the data
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))  # (batch_size, channels, seq_length)
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # Changed shuffle to False for validation

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, loss function, and optimizer
model = LSTMNetwork(input_dim, neurons, layers1, activation, dropout_rate)
model = model.to(device)

# Print trainable parameters
print(f"Trainable parameters: {count_parameters(model)}")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.numel()} parameters")

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
train_losses = []
val_losses = []
min_val_loss = float('inf')


for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for val_X, val_y in test_loader:
            val_X, val_y = val_X.to(device), val_y.to(device)
            val_outputs = model(val_X)
            val_loss += criterion(val_outputs, val_y).item()
    val_loss /= len(test_loader)
    train_losses.append(loss.item())
    val_losses.append(val_loss)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}, Val Loss: {val_loss:.6f}')
    if val_loss < min_val_loss:
        min_val_loss = val_loss


# Print the minimum validation loss
print(f"Minimum Validation Loss (MSE): {min_val_loss:.6f}")

# Compute the RMSE of the minimum validation loss
# Since min_val_loss is in MSE (normalized), we need to convert it to the original scale
# RMSE_original = sqrt(MSE_normalized) * (x_max - x_min)
scale = scaler.data_max_ - scaler.data_min_
rmse_min_val_loss_unnorm = np.sqrt(min_val_loss) * scale
print(f"Unnormalized Root Mean Squared Error of Minimum Validation Loss: {rmse_min_val_loss_unnorm[0]:.6f}")

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(test_X.to(device)).cpu().numpy()

# Inverse transform the scaled predictions and true values to original scale
predictions_unnorm = scaler.inverse_transform(predictions)
test_y_unnorm = scaler.inverse_transform(test_y.numpy())

# Evaluate the model on normalized data
mse_normalized = mean_squared_error(test_y.numpy(), predictions)
rmse_normalized = np.sqrt(mse_normalized)

# Evaluate the model on unnormalized data
mse_unnorm = mean_squared_error(test_y_unnorm, predictions_unnorm)
rmse_unnorm = np.sqrt(mse_unnorm)
mae_unnorm = mean_absolute_error(test_y_unnorm, predictions_unnorm)

print(f"Mean Squared Error (MSE) on Test Set (Normalized): {mse_normalized:.6f}")
print(f"Root Mean Squared Error (RMSE) on Test Set (Normalized): {rmse_normalized:.6f}")
print(f"Mean Squared Error (MSE) on Test Set (Unnormalized): {mse_unnorm:.6f}")
print(f"Root Mean Squared Error (RMSE) on Test Set (Unnormalized): {rmse_unnorm:.6f}")
print(f"Mean Absolute Error (MAE) on Test Set (Unnormalized): {mae_unnorm:.6f}")



torch.Size([21020, 3]) torch.Size([21020, 1]) torch.Size([5252, 3]) torch.Size([5252, 1])
Trainable parameters: 6881
lstm.weight_ih_l0: 384 parameters
lstm.weight_hh_l0: 4096 parameters
lstm.bias_ih_l0: 128 parameters
lstm.bias_hh_l0: 128 parameters
layers.0.weight: 1024 parameters
layers.0.bias: 32 parameters
layers.2.weight: 1024 parameters
layers.2.bias: 32 parameters
output_layer.weight: 32 parameters
output_layer.bias: 1 parameters
Epoch 1/60, Loss: 0.003236, Val Loss: 0.004833
Epoch 2/60, Loss: 0.014932, Val Loss: 0.003983
Epoch 3/60, Loss: 0.015693, Val Loss: 0.003664
Epoch 4/60, Loss: 0.002514, Val Loss: 0.003636
Epoch 5/60, Loss: 0.006996, Val Loss: 0.003550
Epoch 6/60, Loss: 0.008389, Val Loss: 0.003586
Epoch 7/60, Loss: 0.002899, Val Loss: 0.003528
Epoch 8/60, Loss: 0.008473, Val Loss: 0.003407
Epoch 9/60, Loss: 0.005718, Val Loss: 0.003383
Epoch 10/60, Loss: 0.005131, Val Loss: 0.003392
Epoch 11/60, Loss: 0.004763, Val Loss: 0.003678
Epoch 12/60, Loss: 0.002175, Val Loss: 0

In [5]:
print(model)

LSTMNetwork(
  (lstm): LSTM(3, 32, batch_first=True)
  (layers): ModuleList(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.05, inplace=False)
  )
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)
